In [14]:
import numpy as np
import torch
import scipy.linalg as sla
from models import *
from pl_modules.LAModel import LAModel
from utils.deltas import normal
from scipy import sparse
from pathlib import Path

In [23]:
def test_cases(cases, ckpt_path, imgs_save_path, a=1):
    for f in ckpt_path.iterdir():
        print(f)
        # Get all parameters from the ckpt name
        backward_method, n, model_name, features, bs, data_type = f.name.split('_')
        n = int(n)
        layers = int(model_name[-1])
        model_name = model_name[:-1]
        features = int(features)
        bs = int(bs[2:])
        boundary_type = data_type[-1]
        data_type = data_type[:-1]

        h = 2*a / (n -1)
        h2 = h ** 2
        x = np.linspace(-a, a, n)
        y = np.linspace(-a, a, n)
        xx, yy = np.meshgrid(x, y)
        
        # get network
        mat_path = f'./data/{n}/mat/'
        net = model_names[model_name](layers = layers, features = features, boundary_type = boundary_type)
        pl_module = LAModel(net, a, n, data_path = mat_path, backward_type=backward_method, boundary_type=boundary_type)

        # get matrix
        # A = pl_module.A
        lu, piv = sla.lu_factor(pl_module.A.to_dense().numpy())
        
        #get image save path and ckpy file path
        img_path = imgs_save_path/f.name
        ckpt = f/'version_0'/'checkpoints'/'last.ckpt'

        # Load ckpt
        ckpt = torch.load(ckpt)
        pl_module.load_state_dict(ckpt['state_dict'])
        pl_module.freeze()

        # Test all cases
        for case in test_cases:
            f = np.zeros((n, n))
            for i in case:
                px, py, q = i
                f += q * normal(xx, yy, h, (px, py))
            
                # Get input tensor for networks
                input_tensor = np.stack([xx, yy, f], axis=0)
                input_tensor = torch.from_numpy(input_tensor).float()
            
                # Get b for linear equations, If the value of boundary changed should fix here
                b = f.reshape(n**2)
                # b = apply_boundary(b, gn, gd)

                # get predicted value and real ans
                ans = sla.lu_solve((lu, piv), b)


cases = [
    [(0, 0, 1)],
    [(-0.5, -0.5, 1), (0.5, 0.5, 1)],
]
ckpt_path = Path('./lightning_logs/')
imgs_save_path = Path('./images/')
test_cases(cases, ckpt_path, imgs_save_path)

lightning_logs/cg_64_varyUNet4_16_bs16_mixedN
